In [1]:
# Install dependencies
!pip install fpdf2 gradio pillow

# Imports
from fpdf import FPDF
import datetime
import gradio as gr
from PIL import Image

# Resume PDF Class
class StylishResume(FPDF):
    def __init__(self, photo_path=None):
        super().__init__()
        self.photo_path = photo_path
        self.set_auto_page_break(auto=True, margin=15)
        self.add_page()
        self.set_margins(15, 15, 15)

    def header(self):
        self.set_fill_color(255, 230, 240)  # pastel pink background
        self.rect(0, 0, 210, 40, 'F')
        if self.photo_path:
            self.image(self.photo_path, 170, 5, 30, 30)
        self.set_font('Helvetica', 'B', 20)
        self.set_text_color(70, 40, 60)
        self.set_xy(15, 12)
        self.cell(0, 10, "Professional Resume", ln=True)

    def add_personal_details(self, name, email, phone, address, summary):
        self.ln(35)
        self.set_font("Helvetica", "B", 14)
        self.set_text_color(80, 30, 60)
        self.cell(0, 10, name, ln=True)

        self.set_font("Helvetica", "", 12)
        self.set_text_color(60, 60, 60)
        self.cell(0, 8, f"Email: {email}", ln=True)
        self.cell(0, 8, f"Phone: {phone}", ln=True)
        self.multi_cell(0, 8, f"Address: {address}")
        self.ln(5)

        self.set_fill_color(255, 210, 230)
        self.set_font("Helvetica", "B", 13)
        self.set_text_color(60, 20, 50)
        self.cell(0, 10, "Career Summary", ln=True, fill=True)

        self.set_font("Helvetica", "", 12)
        self.set_text_color(50, 50, 50)
        self.multi_cell(0, 8, summary)
        self.ln(5)

    def add_section(self, title, content):
        self.set_fill_color(255, 220, 235)
        self.set_font("Helvetica", "B", 13)
        self.set_text_color(60, 30, 70)
        self.cell(0, 10, title, ln=True, fill=True)

        self.set_font("Helvetica", "", 12)
        self.set_text_color(50, 50, 50)
        self.multi_cell(0, 8, content)
        self.ln(3)

# Cover Letter Class
class CoverLetterPDF(FPDF):
    def __init__(self):
        super().__init__()
        self.set_auto_page_break(auto=True, margin=15)
        self.add_page()
        self.set_margins(20, 20, 20)

    def header(self):
        self.set_fill_color(255, 230, 240)
        self.rect(0, 0, 210, 25, 'F')
        self.set_font('Helvetica', 'B', 14)
        self.set_text_color(70, 30, 50)
        self.cell(0, 8, 'Cover Letter', ln=True, align='C')
        self.ln(10)

    def add_body(self, date_str, recipient_name, company_name, position, body_text, applicant_name, contact_info):
        self.set_font('Helvetica', '', 12)
        self.set_text_color(50, 50, 50)
        self.cell(0, 6, date_str, ln=True)
        self.ln(4)
        self.multi_cell(0, 6, f"{recipient_name}\n{company_name}\n")
        self.ln(4)

        self.set_font('Helvetica', 'B', 12)
        self.set_text_color(30, 30, 70)
        self.cell(0, 6, f"Subject: Application for {position}", ln=True)
        self.ln(6)

        self.set_font('Helvetica', '', 12)
        self.set_text_color(50, 50, 50)
        self.multi_cell(0, 6, body_text)
        self.ln(12)

        self.multi_cell(0, 6, f"Sincerely,\n{applicant_name}\n{contact_info}")

# Resume Creation
def create_resume(name, email, phone, address, summary, education, experience, skills, photo):
    photo_path = None
    if photo is not None:
        img = Image.open(photo.name)
        img = img.convert("RGB")
        photo_path = "/content/profile_photo.jpg"
        img.save(photo_path)

    pdf = StylishResume(photo_path=photo_path)
    pdf.add_personal_details(name, email, phone, address, summary)
    pdf.add_section("Education", education)
    pdf.add_section("Experience", experience)
    pdf.add_section("Skills", skills)

    output_path = "/content/Beautiful_Resume.pdf"
    pdf.output(output_path)
    return output_path

# Cover Letter Creation
def create_cover_letter_pdf(applicant_name, recipient_name, company_name, position, intro, accomplishments, fit_statement, closing_note, contact_info):
    date_str = datetime.datetime.now().strftime('%B %d, %Y')
    body_paragraphs = [intro, accomplishments, fit_statement, closing_note]
    body_text = "\n\n".join([p.strip() for p in body_paragraphs if p and p.strip()])

    pdf = CoverLetterPDF()
    pdf.add_body(date_str=date_str,
                 recipient_name=recipient_name,
                 company_name=company_name,
                 position=position,
                 body_text=body_text,
                 applicant_name=applicant_name,
                 contact_info=contact_info)

    output_path = '/content/Cover_Letter.pdf'
    pdf.output(output_path)
    return output_path

# Gradio App
with gr.Blocks(theme=gr.themes.Soft(primary_hue="pink", secondary_hue="pink", neutral_hue="pink")) as demo:
    gr.Markdown("## 💖 Resume & Cover Letter Generator\nEasily create beautiful PDF documents in pastel pink style.")

    with gr.Tab("📄 Resume Builder"):
        with gr.Row():
            with gr.Column():
                name = gr.Textbox(label="Full Name")
                email = gr.Textbox(label="Email")
                phone = gr.Textbox(label="Phone")
                address = gr.Textbox(label="Address")
                photo = gr.File(label="Upload Profile Photo", file_types=[".jpg", ".jpeg", ".png"])
            with gr.Column():
                summary = gr.Textbox(label="Career Summary", lines=4)
                education = gr.Textbox(label="Education", lines=4)
                experience = gr.Textbox(label="Work Experience", lines=4)
                skills = gr.Textbox(label="Skills", lines=3)

        generate_resume_btn = gr.Button("🎀 Generate Resume")
        resume_output = gr.File(label="⬇️ Download Resume PDF")
        generate_resume_btn.click(fn=create_resume,
                                  inputs=[name, email, phone, address, summary, education, experience, skills, photo],
                                  outputs=resume_output)

    with gr.Tab("✉️ Cover Letter Generator"):
        with gr.Row():
            with gr.Column():
                applicant_name = gr.Textbox(label="Your Name")
                contact_info = gr.Textbox(label="Contact Info (Email / Phone)")
                recipient_name = gr.Textbox(label="Hiring Manager's Name")
                company_name = gr.Textbox(label="Company Name")
                position = gr.Textbox(label="Job Title / Position")
            with gr.Column():
                intro = gr.Textbox(label="Intro Paragraph", lines=2)
                accomplishments = gr.Textbox(label="Accomplishments / Experience", lines=3)
                fit_statement = gr.Textbox(label="Why You Are a Good Fit", lines=2)
                closing_note = gr.Textbox(label="Closing Note", lines=2)

        generate_cover_btn = gr.Button("🎀 Generate Cover Letter")
        cover_output = gr.File(label="⬇️ Download Cover Letter PDF")
        generate_cover_btn.click(fn=create_cover_letter_pdf,
                                 inputs=[applicant_name, recipient_name, company_name, position, intro, accomplishments, fit_statement, closing_note, contact_info],
                                 outputs=cover_output)

demo.launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 5.1 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b18972c68145e47b04.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
